## Iteración 1:

Este script emula el funcionamiento diario de un parqueadero bajo sus principales procesos:

* **Operario**: almacena la información de quien supervisa el turno (cédula, nombre, correo y rol).
* **Celdas**: dispone de 10 espacios para carros y 6 para motos; cada una lleva registro de su tipo y estado (ocupada o libre).
* **Vehículos**: se registran con placa, tipo (carro o moto), marca y color.
* **Tiquete de ingreso**: al aparcar, se genera un comprobante que incluye los datos del vehículo, la celda asignada y la hora de entrada.
* **Tiquete de salida**: al retirar el vehículo, el comprobante muestra la hora de salida, el método de pago y el monto a pagar (tarifa por hora completa: 3 500 COP para carros y 2 000 COP para motos).
* **Estado del parqueadero**: después de cada ingreso o salida, se actualiza y muestra cuántas celdas de carro y de moto están ocupadas y cuántas están disponibles.
* **Flujo de la simulación**: en `main()` se presenta el estado inicial, se procesan múltiples ingresos (incluyendo una placa duplicada que genera alerta) y varias salidas con diferentes métodos de pago.


## Iteración 2:

El sistema ahora permite una gestión completa de usuarios y sus vehículos. Cuando un vehículo ingresa al parqueadero:

*   **Registro de Usuario:** Se solicita información del usuario (nombre, cédula, correo y teléfono) que se almacena pero no se muestra en los tiquetes
*   **Tiquetes Simplificados:** Los tiquetes solo muestran el nombre del usuario, manteniendo privados los demás datos personales.
* **Historial de Visitas:** El operario puede consultar el historial completo de cualquier usuario, viendo todas sus visitas al parqueadero con detalles como fechas, horas, vehículos usados y métodos de pago.
* **Gestión de Celdas:** Cada celda mantiene un registro de los vehículos que la han ocupado, permitiendo analizar patrones de uso.
* **Interfaz Amigable:** Menú interactivo que guía al operario por todas las funciones del sistema.
* **Datos Persistentes:** Toda la información queda registrada durante la ejecución del programa, permitiendo consultas históricas.

In [ ]:
import time
from datetime import datetime, timedelta
from collections import defaultdict

class User:
    def __init__(self, id_number, full_name, email, phone):
        self.id_number = id_number
        self.full_name = full_name
        self.email = email
        self.phone = phone
        self.vehicles = []

    def add_vehicle(self, vehicle):
        self.vehicles.append(vehicle)
        vehicle.owner = self

    def get_visit_history(self):
        return [ticket for vehicle in self.vehicles
                for ticket in vehicle.visit_history]

class Operator:
    def __init__(self, id_number, full_name, email, role):
        self.id_number = id_number
        self.full_name = full_name
        self.email = email
        self.role = role

class Cell:
    def __init__(self, cell_id, cell_type):
        self.cell_id = cell_id
        self.cell_type = cell_type  # 'carro' o 'moto'
        self.occupied = False
        self.vehicle = None
        self.usage_history = []

class Vehicle:
    def __init__(self, license_plate, vehicle_type, brand, color, owner=None):
        self.license_plate = license_plate
        self.vehicle_type = vehicle_type  # 'carro' o 'moto'
        self.brand = brand
        self.color = color
        self.owner = owner
        self.visit_history = []

class Ticket:
    def __init__(self, vehicle, cell, entry_time):
        self.vehicle = vehicle
        self.cell = cell
        self.entry_time = entry_time
        self.exit_time = None
        self.paid = False
        self.payment_method = None
        # Registrar en historiales
        vehicle.visit_history.append(self)
        cell.usage_history.append(self)
        if vehicle.owner:
            vehicle.owner.get_visit_history().append(self)

    def calculate_fee(self):
        if not self.exit_time:
            return 0
        rate = 3500 if self.vehicle.vehicle_type == 'carro' else 2000
        total_seconds = (self.exit_time - self.entry_time).total_seconds()
        hours = int(total_seconds // 3600)
        minutes = int((total_seconds % 3600) // 60)
        # cobramos al menos 1 hora, y cada minuto extra cuenta como hora completa
        hours += 1 if minutes > 0 else 0
        hours = max(1, hours)
        return hours * rate

    def permanence(self):
        if not self.exit_time:
            return "00:00"
        delta = self.exit_time - self.entry_time
        total_minutes = int(delta.total_seconds() // 60)
        h, m = divmod(total_minutes, 60)
        return f"{h:02d}:{m:02d}"

    def pretty_print(self, exit=False):
        title = "TIQUETE DE SALIDA" if exit else "TIQUETE DE INGRESO"
        entry_str = self.entry_time.strftime('%H:%M')
        print(f"\n—— {title} ——")
        print(f"Tipo Vehículo : {self.vehicle.vehicle_type.capitalize()}")
        print(f"Placa         : {self.vehicle.license_plate}")
        print(f"Marca         : {self.vehicle.brand}")
        print(f"Color         : {self.vehicle.color}")
        if self.vehicle.owner:
            print(f"Usuario       : {self.vehicle.owner.full_name}")
        print(f"Celda         : {self.cell.cell_id} ({self.cell.cell_type})")
        print(f"Entrada       : {entry_str}")
        if exit:
            exit_str = self.exit_time.strftime('%H:%M')
            fee = self.calculate_fee()
            print(f"Salida        : {exit_str}")
            print(f"Permanencia   : {self.permanence()}")
            print(f"Método Pago   : {self.payment_method}")
            print(f"Total a Pagar : {fee} COP")
        print("———————————————\n")

class ParkingLot:
    def __init__(self, operator, car_cells=10, moto_cells=6):
        self.operator = operator
        self.cells = []
        for i in range(car_cells):
            self.cells.append(Cell(i+1, 'carro'))
        for j in range(moto_cells):
            self.cells.append(Cell(car_cells + j+1, 'moto'))
        self.tickets = {}
        self.users = {}
        self.vehicles = {}

    def available_cells(self, vehicle_type=None):
        cells = [c for c in self.cells if not c.occupied]
        if vehicle_type:
            cells = [c for c in cells if c.cell_type == vehicle_type]
        return cells

    def occupied_cells(self):
        return [c for c in self.cells if c.occupied]

    def report_status(self):
        cars_free = self.available_cells('carro')
        motos_free = self.available_cells('moto')
        cars_occ = [c for c in self.occupied_cells() if c.cell_type == 'carro']
        motos_occ = [c for c in self.occupied_cells() if c.cell_type == 'moto']
        print("\nEstado actual del parqueadero:")
        print(f"  Celdas disponibles para CARRO ({len(cars_free)}) : {[c.cell_id for c in cars_free]}")
        print(f"  Celdas ocupadas para CARRO  ({len(cars_occ)}) : {[c.cell_id for c in cars_occ]}")
        print(f"  Celdas disponibles para MOTO  ({len(motos_free)}) : {[c.cell_id for c in motos_free]}")
        print(f"  Celdas ocupadas para MOTO   ({len(motos_occ)}) : {[c.cell_id for c in motos_occ]}")
        print()

    def register_user(self, id_number, full_name, email, phone):
        if id_number in self.users:
            print(f"Usuario con cédula {id_number} ya está registrado.")
            return self.users[id_number]
        user = User(id_number, full_name, email, phone)
        self.users[id_number] = user
        print(f"✅ Usuario {full_name} registrado exitosamente.")
        return user

    def register_vehicle(self, license_plate, vehicle_type, brand, color, owner=None):
        if license_plate in self.vehicles:
            print(f"Vehículo con placa {license_plate} ya está registrado.")
            return self.vehicles[license_plate]
        vehicle = Vehicle(license_plate, vehicle_type, brand, color, owner)
        self.vehicles[license_plate] = vehicle
        if owner:
            owner.add_vehicle(vehicle)
        print(f"✅ Vehículo {license_plate} registrado exitosamente.")
        return vehicle

    def park_vehicle(self, vehicle, entry_time=None):
        if vehicle.license_plate in self.tickets:
            print(f"⚠️ Alerta: La placa {vehicle.license_plate} ya está registrada en el parqueadero.")
            return None
        free = self.available_cells(vehicle.vehicle_type)
        if not free:
            print(f"No hay celdas disponibles para {vehicle.vehicle_type}.")
            return None
        cell = free[0]
        cell.occupied = True
        cell.vehicle = vehicle
        entry = entry_time or datetime.now()
        ticket = Ticket(vehicle, cell, entry)
        self.tickets[vehicle.license_plate] = ticket
        print(f"✅ Registro exitoso para la placa {vehicle.license_plate}.")
        ticket.pretty_print(exit=False)
        self.report_status()
        return ticket

    def release_vehicle(self, license_plate, exit_time=None, payment_method='efectivo'):
        ticket = self.tickets.get(license_plate)
        if not ticket:
            print(f"⚠️ Ticket no encontrado para {license_plate}.")
            return
        ticket.exit_time = exit_time or datetime.now()
        ticket.paid = True
        ticket.payment_method = payment_method
        cell = ticket.cell
        cell.occupied = False
        cell.vehicle = None
        ticket.pretty_print(exit=True)
        self.report_status()
        del self.tickets[license_plate]

    def user_history(self, user_id):
        user = self.users.get(user_id)
        if not user:
            print(f"Usuario con cédula {user_id} no encontrado.")
            return
        print(f"\nHistorial de visitas para {user.full_name} (Cédula: {user.id_number}):")
        visits = sorted(user.get_visit_history(), key=lambda x: x.entry_time, reverse=True)
        if not visits:
            print("  No hay registros de visitas.")
            return
        for i, visit in enumerate(visits, 1):
            vehicle = visit.vehicle
            print(f"\n  Visita #{i}:")
            print(f"  Vehículo: {vehicle.brand} {vehicle.color} - Placa: {vehicle.license_plate}")
            print(f"  Fecha entrada: {visit.entry_time.strftime('%Y-%m-%d %H:%M')}")
            if visit.exit_time:
                print(f"  Fecha salida: {visit.exit_time.strftime('%Y-%m-%d %H:%M')}")
                print(f"  Permanencia: {visit.permanence()}")
                print(f"  Método pago: {visit.payment_method}")
                print(f"  Total pagado: {visit.calculate_fee()} COP")
            else:
                print("  ⚠️ Vehículo aún en el parqueadero")

    def cell_history(self, cell_id):
        cell = next((c for c in self.cells if c.cell_id == cell_id), None)
        if not cell:
            print(f"Celda {cell_id} no encontrada.")
            return
        print(f"\nHistorial de uso para Celda {cell_id} ({cell.cell_type}):")
        if not cell.usage_history:
            print("  No hay registros de uso.")
            return
        for i, usage in enumerate(sorted(cell.usage_history, key=lambda x: x.entry_time, reverse=True), 1):
            print(f"\n  Uso #{i}:")
            vehicle = usage.vehicle
            print(f"  Vehículo: {vehicle.license_plate} ({vehicle.vehicle_type})")
            if vehicle.owner:
                print(f"  Usuario: {vehicle.owner.full_name} (Cédula: {vehicle.owner.id_number})")
            print(f"  Fecha entrada: {usage.entry_time.strftime('%Y-%m-%d %H:%M')}")
            if usage.exit_time:
                print(f"  Fecha salida: {usage.exit_time.strftime('%Y-%m-%d %H:%M')}")
                print(f"  Permanencia: {usage.permanence()}")
            else:
                print("  ⚠️ Vehículo aún en esta celda")

    def show_operator_menu(self):
        print("\nMenú del Operario:")
        print("1. Registrar ingreso de vehículo")
        print("2. Registrar salida de vehículo")
        print("3. Consultar historial de usuario")
        print("4. Consultar historial de celda")
        print("5. Ver estado actual del parqueadero")
        print("6. Salir")
        choice = input("Seleccione una opción: ")
        return choice

# Simulación interactiva mejorada

def interactive_simulation():
    print("\n" + "="*50)
    print(" SISTEMA DE GESTIÓN DE PARQUEADERO ".center(50, '='))
    print("="*50 + "\n")

    # Registrar operario
    print("\nRegistro del operario:")
    op_id = input("Cédula del operario: ")
    op_name = input("Nombre completo: ")
    op_email = input("Correo electrónico: ")
    op_role = input("Rol: ")

    operario = Operator(op_id, op_name, op_email, op_role)
    print(f"\nOperario registrado: {operario.full_name} ({operario.role})")

    lot = ParkingLot(operator=operario)
    lot.report_status()

    while True:
        choice = lot.show_operator_menu()

        if choice == '1':  # Registrar ingreso
            print("\nRegistro de vehículo entrante:")

            # Verificar si es usuario registrado
            user_choice = input("¿El usuario está registrado? (s/n): ").lower()
            user = None
            if user_choice == 's':
                user_id = input("Cédula del usuario: ")
                user = lot.users.get(user_id)
                if not user:
                    print("Usuario no encontrado. Registre al usuario primero.")
                    continue
            else:
                print("\nRegistro de nuevo usuario:")
                user_id = input("Cédula: ")
                user_name = input("Nombre completo: ")
                user_email = input("Correo electrónico: ")
                user_phone = input("Teléfono: ")
                user = lot.register_user(user_id, user_name, user_email, user_phone)

            # Registrar vehículo
            print("\nDatos del vehículo:")
            plate = input("Placa: ").upper()
            type_ = input("Tipo (carro/moto): ").lower()
            while type_ not in ['carro', 'moto']:
                print("Tipo inválido. Debe ser 'carro' o 'moto'")
                type_ = input("Tipo (carro/moto): ").lower()
            brand = input("Marca: ")
            color = input("Color: ")

            vehicle = lot.register_vehicle(plate, type_, brand, color, user)
            lot.park_vehicle(vehicle)

        elif choice == '2':  # Registrar salida
            print("\nRegistro de vehículo saliente:")
            plate = input("Placa del vehículo: ").upper()
            method = input("Método de pago (efectivo/tarjeta/transferencia): ").lower()
            while method not in ['efectivo', 'tarjeta', 'transferencia']:
                print("Método inválido. Debe ser 'efectivo', 'tarjeta' o 'transferencia'")
                method = input("Método de pago: ").lower()
            lot.release_vehicle(plate, payment_method=method)

        elif choice == '3':  # Historial usuario
            user_id = input("\nCédula del usuario a consultar: ")
            lot.user_history(user_id)

        elif choice == '4':  # Historial celda
            try:
                cell_id = int(input("\nID de la celda a consultar: "))
                lot.cell_history(cell_id)
            except ValueError:
                print("ID de celda debe ser un número.")

        elif choice == '5':  # Estado actual
            lot.report_status()

        elif choice == '6':  # Salir
            print("\nSaliendo del sistema...")
            break

        else:
            print("\nOpción inválida. Intente nuevamente.")

        input("\nPresione Enter para continuar...")

if __name__ == "__main__":
    interactive_simulation()